# S&P 500 Backtesting - Exploratory Data Analysis

This notebook performs exploratory data analysis on the S&P 500 stock price data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add scripts directory to path
sys.path.append('../scripts')

from memory_reducer import memory_reducer

plt.style.use('default')
sns.set_palette("husl")

%matplotlib inline

## 1. Data Loading and Memory Optimization

In [ ]:
# Load data with memory optimization
prices = memory_reducer('../data/prices.csv')
sp500 = memory_reducer('../data/sp500.csv')

print("Prices data shape:", prices.shape)
print("S&P 500 data shape:", sp500.shape)
print("\nPrices columns:", prices.columns.tolist())
print("S&P 500 columns:", sp500.columns.tolist())

## 2. Missing Values Analysis

In [ ]:
# Missing values in prices data
print("Missing values in prices data:")
print(prices.isnull().sum())
print(f"\nTotal missing values: {prices.isnull().sum().sum()}")
print(f"Percentage of missing values: {(prices.isnull().sum().sum() / prices.size) * 100:.2f}%")

# Missing values in S&P 500 data
print("\nMissing values in S&P 500 data:")
print(sp500.isnull().sum())

In [ ]:
# Visualize missing values pattern
plt.figure(figsize=(12, 6))
prices_sample = prices.sample(n=min(10000, len(prices)))  # Sample for visualization
sns.heatmap(prices_sample.isnull(), cbar=True, yticklabels=False)
plt.title('Missing Values Pattern in Stock Prices Data (Sample)')
plt.tight_layout()
plt.savefig('../results/plots/missing_values_heatmap.png')
plt.show()

## 3. Outliers Analysis

In [ ]:
# Price distribution analysis
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(prices['price'].dropna(), bins=100, alpha=0.7)
plt.title('Price Distribution')
plt.xlabel('Price ($)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(np.log(prices['price'].dropna() + 1), bins=100, alpha=0.7)
plt.title('Log Price Distribution')
plt.xlabel('Log(Price + 1)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.boxplot(prices['price'].dropna())
plt.title('Price Box Plot')
plt.ylabel('Price ($)')

plt.tight_layout()
plt.savefig('../results/plots/price_distribution_analysis.png')
plt.show()

print(f"Price statistics:")
print(prices['price'].describe())

In [ ]:
# Identify extreme outliers
price_q99 = prices['price'].quantile(0.99)
price_q01 = prices['price'].quantile(0.01)

extreme_high = prices[prices['price'] > price_q99]
extreme_low = prices[prices['price'] < price_q01]

print(f"Extreme high prices (>99th percentile, ${price_q99:.2f}):")
print(extreme_high.nlargest(10, 'price')[['ticker', 'date', 'price']])

print(f"\nExtreme low prices (<1st percentile, ${price_q01:.2f}):")
print(extreme_low.nsmallest(10, 'price')[['ticker', 'date', 'price']])

## 4. Price Consistency Analysis

In [ ]:
# Convert date to datetime for analysis
prices['date'] = pd.to_datetime(prices['date'])

# Average price over time
avg_price_over_time = prices.groupby('date')['price'].mean()

plt.figure(figsize=(15, 6))
plt.plot(avg_price_over_time.index, avg_price_over_time.values)
plt.title('Average Stock Price Over Time')
plt.xlabel('Date')
plt.ylabel('Average Price ($)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/plots/average_price_over_time.png')
plt.show()

# Average price by company
company_avg_prices = prices.groupby('ticker')['price'].mean().sort_values(ascending=False)

plt.figure(figsize=(15, 8))
plt.bar(range(len(company_avg_prices)), company_avg_prices.values)
plt.title('Average Price for Each Company in Dataset')
plt.xlabel('Company (Ticker)')
plt.ylabel('Average Price ($)')
plt.xticks(range(0, len(company_avg_prices), max(1, len(company_avg_prices)//10)), 
           company_avg_prices.index[::max(1, len(company_avg_prices)//10)], rotation=45)
plt.tight_layout()
plt.savefig('../results/plots/average_price_by_company.png')
plt.show()

In [ ]:
# Price consistency across companies
company_stats = prices.groupby('ticker')['price'].agg(['mean', 'std', 'count']).reset_index()
company_stats['cv'] = company_stats['std'] / company_stats['mean']  # Coefficient of variation

plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.hist(company_stats['mean'], bins=50, alpha=0.7)
plt.title('Distribution of Average Prices by Company')
plt.xlabel('Average Price ($)')
plt.ylabel('Number of Companies')

plt.subplot(2, 2, 2)
plt.hist(company_stats['cv'], bins=50, alpha=0.7)
plt.title('Distribution of Price Volatility (CV) by Company')
plt.xlabel('Coefficient of Variation')
plt.ylabel('Number of Companies')

plt.subplot(2, 2, 3)
plt.scatter(company_stats['mean'], company_stats['std'], alpha=0.6)
plt.title('Price Mean vs Standard Deviation')
plt.xlabel('Average Price ($)')
plt.ylabel('Price Standard Deviation')

plt.subplot(2, 2, 4)
plt.hist(company_stats['count'], bins=50, alpha=0.7)
plt.title('Distribution of Data Points per Company')
plt.xlabel('Number of Data Points')
plt.ylabel('Number of Companies')

plt.tight_layout()
plt.savefig('../results/plots/price_consistency_analysis.png')
plt.show()

print("Top 10 most volatile companies (by CV):")
print(company_stats.nlargest(10, 'cv')[['ticker', 'mean', 'std', 'cv']])

## 5. Outlier Documentation

In [ ]:
# Identify and document specific outliers
outliers_list = []

# Method 1: Statistical outliers using IQR
for ticker in prices['ticker'].unique()[:50]:  # Sample first 50 tickers
    ticker_data = prices[prices['ticker'] == ticker]
    if len(ticker_data) > 10:  # Only analyze tickers with sufficient data
        Q1 = ticker_data['price'].quantile(0.25)
        Q3 = ticker_data['price'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = ticker_data[(ticker_data['price'] < lower_bound) | 
                              (ticker_data['price'] > upper_bound)]
        
        for _, row in outliers.head(2).iterrows():  # Take top 2 outliers per ticker
            outliers_list.append({
                'ticker': row['ticker'],
                'date': row['date'].strftime('%Y-%m-%d'),
                'price': row['price']
            })

# Display first 5 outliers
print("Top 5 identified outliers:")
for i, outlier in enumerate(outliers_list[:5]):
    print(f"{i+1}. Ticker: {outlier['ticker']}, Date: {outlier['date']}, Price: ${outlier['price']:.4f}")

# Save to file
os.makedirs('../results', exist_ok=True)
with open('../results/outliers.txt', 'w') as f:
    f.write("ticker,date,price\n")
    for outlier in outliers_list[:5]:
        f.write(f"{outlier['ticker']},{outlier['date']},{outlier['price']:.4f}\n")

print("\nOutliers saved to ../results/outliers.txt")

## 6. Data Quality Summary

In [ ]:
print("DATA QUALITY SUMMARY")
print("====================")
print(f"Total records: {len(prices):,}")
print(f"Unique tickers: {prices['ticker'].nunique()}")
print(f"Date range: {prices['date'].min()} to {prices['date'].max()}")
print(f"Missing values: {prices.isnull().sum().sum():,} ({(prices.isnull().sum().sum() / prices.size) * 100:.2f}%)")
print(f"Price range: ${prices['price'].min():.4f} to ${prices['price'].max():.2f}")
print(f"Median price: ${prices['price'].median():.2f}")
print(f"Potential outliers identified: {len(outliers_list)}")

print("\nKey data quality issues identified:")
print("- Significant missing values requiring forward-fill strategy")
print("- Extreme price outliers requiring filtering")
print("- Price spikes that may indicate stock splits or data errors")
print("- Varying data availability across different tickers")